In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
import datetime
import sys
import os
from optimization import v2gOptimization

In [ ]:
color_palette = ['#332288',  '#117733', '#44AA99', '#88CCEE', '#DDCC77', '#CC6677', '#AA4499', '#882255']
parent_dir = os.path.dirname(os.getcwd())
path = parent_dir 
save_path = parent_dir + '/'

# Figure S1: weekly dynamic prices 

In [ ]:
def plot_accessories(ax):
    #set axis ticks and labels to days
    ax.set_xticks(np.arange(0, 7*24*60, 24*60))
    ax.set_xticklabels(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])  

    ax.xaxis.set_tick_params(labelsize=13)
    ax.yaxis.set_tick_params(labelsize=13)
    ax.set_xlabel('Time [day]', fontsize=14)
    ax.set_ylabel('Price [$/kWh]', fontsize=14)

In [ ]:
fig, ax = plt.subplots(4,1,figsize=(12, 16))
plt.subplots_adjust(hspace=0.25)
plt.subplots_adjust(wspace=0.2)

ax = ax.flatten()
fig_label = ['a.', 'b.', 'c.', 'd.']

batt_params_file = 'Parameters/params_battery_model.json'

for k, circuit in enumerate(['1_cir', '2_cir', '3_cir', '4_cir']):
    params_filename = 'Parameters/params_pge_0_batt_'+circuit+'.json'
    v2g_opt = v2gOptimization(params_filename, batt_params_file)

    mondays, num_weeks = v2g_opt.define_weeks(v2g_opt.min_date_price, v2g_opt.max_date_price, v2g_opt.prices)
    for j, monday in enumerate(mondays[15:20]):
        start = monday
        period_string = str(start) + '_to_' + str(start + datetime.timedelta(days=6))

        #get the dynamic price vector for the price week
        price_week = v2g_opt.prices.loc[(v2g_opt.prices.datetime.dt.date >= start)&(v2g_opt.prices.datetime.dt.date <= start + datetime.timedelta(days=6))].Price.values
        price_week = np.repeat(price_week / 100, 60)

        #set up the ELRP price vector for the price week
        tmp_elrp = price_week
        #pick 10 random days to be ELRP days
        np.random.seed(1)
        elrp_days = np.random.choice(np.arange(121,v2g_opt.elrp_total_num_days+121), 10, replace=False)
        elrp_start_times = np.random.choice([16, 17, 18], 10, replace=True)
        #loop through each day to check if it's an ELRP day
        for i in range(7):
            day = start + datetime.timedelta(days=i)
            if day.timetuple().tm_yday in elrp_days:
                start_time = i*24*60 + elrp_start_times[np.where(elrp_days==day.timetuple().tm_yday)[0][0]]*60
                tmp_elrp[start_time:start_time + 3 * 60 ] = 2
        price_week_elrp = tmp_elrp
        ax[k].plot(price_week_elrp,  color = color_palette[j], label = str(start))

    plot_accessories(ax[k])
    ax[k].annotate('Circuit ' + str(k+1), xy=(0.5, .9), xycoords='axes fraction', ha='center', fontsize=16)
    ax[k].annotate(fig_label[k], xy=(-.05, 1.05), xycoords='axes fraction', ha='center', fontsize=16)

plt.savefig(save_path + 'S1_price_weeks.pdf', bbox_inches='tight')